In [ ]:
import sys
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.legend_handler import HandlerPatch
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import tensorflow as tf


In [6]:
def extract_star_predictions(prediction, threshold=0.4):
    # Normalize the prediction array to be between 0 and 1
    prediction = (prediction - prediction.min()) / (prediction.max() - prediction.min())

    # Ensure the prediction array is 2D
    if prediction.ndim == 3:
        prediction = prediction[:, :, 0]

    # Threshold the prediction array to get the star locations
    stars = np.argwhere(prediction > threshold)

    # Create a list to store the star data
    star_data = []

    # Create a prediction mask of the same shape as the prediction array
    prediction_mask = np.zeros_like(prediction, dtype=np.uint8)

    # Iterate over the star locations and add them to the star data list and prediction mask
    for star in stars:
        y, x = star
        star_data.append((x, y))
        prediction_mask[y, x] = 1

    return star_data, prediction_mask

In [7]:
def plot_image_mask_prediction_overlay(image, test_image, model, threshold=0.4):
    pred_star_data, prediction_mask = extract_star_predictions(model.predict(np.expand_dims(test_image, axis=0))[0], threshold=threshold)
    print(np.count_nonzero(prediction_mask))


    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)

    # Plot the image
    ax.imshow(image, cmap='gray', origin='lower')

    # Draw red circles on the image for star predictions
    x_dim, y_dim = image.shape

    # Pixel-mask of stars
    pixel_mask = np.zeros((x_dim, y_dim))

    print('Drawing')  # DEBUG

    for star in pred_star_data:
        pixel_coords = star
        # Ensure the pixel coordinates are within bounds
        x, y = int(np.round(pixel_coords[0])), int(np.round(pixel_coords[1]))
        if 0 <= x < x_dim and 0 <= y < y_dim:
            pixel_mask[x][y] = 1

        Drawing_colored_circle = plt.Circle((pixel_coords[0], pixel_coords[1]), 4, fill=False, edgecolor='red', linewidth=0.2)
        ax.add_artist(Drawing_colored_circle)

    ax.set_title(f'{"Image with Star Prediction Overlay"}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.grid(color='white', ls='dotted')

    # Add legend
    def make_legend_circle(legend, orig_handle, xdescent, ydescent, width, height, fontsize):
        return Circle((width / 2, height / 2), 0.25 * height, fill=False, edgecolor=orig_handle.get_edgecolor(), linewidth=orig_handle.get_linewidth())


    # Display a legend for the circles
    blue_circle = Circle((0, 0), 1, fill=False, edgecolor='blue', linewidth=1)
    red_circle = Circle((0, 0), 1, fill=False, edgecolor='red', linewidth=1)
    ax.legend([blue_circle, red_circle], ['Star Location', 'Star Prediction'], loc='upper right', handler_map={Circle: HandlerPatch(patch_func=make_legend_circle)})


    plt.show()

In [13]:
image_path = "image.png"

# Load the image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


In [ ]:
# Load the trained model
saved_models_path = '../models/saved_models/'
model_files = [file for file in os.listdir(saved_models_path) if file.endswith('.keras')]
model_files = sorted(model_files)
model_choice = model_files[-1]
model = load_model(os.path.join(saved_models_path, model_choice))

In [14]:

# Expand dimensions to match the model input
stacked_image = np.array(np.stack([image, image, image], axis=-1))
image_input = np.array(stacked_image)


In [15]:

# Check for GPU availability
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU is available")
    K.clear_session()
    tf.config.experimental.reset_memory_stats('GPU:0')

# Make prediction
prediction = model.predict(image_input)

# Extract star predictions
star_data, prediction_mask = extract_star_predictions(prediction)

# Find contours of the prediction mask
contours, _ = cv2.findContours(prediction_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw red circles around the predicted areas on the original image
output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for contour in contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    radius = int(radius)
    cv2.circle(output_image, center, radius, (0, 0, 255), 2)

# Save the output image
output_path = os.path.splitext(image_path)[0] + '_prediction.png'
cv2.imwrite(output_path, output_image)
print(f"Prediction saved to {output_path}")

# Display the input image and the output image with predictions
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Input Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(output_image)
plt.title('Prediction Overlay')
plt.axis('off')

plt.show()

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python demoModel.py <path_to_image>")
    else:
        main(sys.argv[1])

GPU is available


ValueError: in user code:

    File "/home/chris/github/AstrID/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/chris/github/AstrID/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/chris/github/AstrID/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/chris/github/AstrID/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/chris/github/AstrID/.venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/chris/github/AstrID/.venv/lib/python3.10/s